<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/testing_all_the_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#overview
I'm assuming everything in all-the-news is mostly true. It doesn't matter much for this model anyways since I am aiming to have it actively scrap relevant articles. I'll later figure out how to get adversarial examples (must indicate true news as fake when all given articles are fake; does this stop the model from picking on finer details like word usage?).

# Conclusion
This dataset is pretty bad. It will take a lot of cleaning to use.... I would still have used this datset though but I have found something better. Not exactly a dataset, but a modern newspaper scraper that can very accurately parse most mainstream news sources, including author, category, etc. I will be using that...

In [0]:
import sqlite3
import pandas as pd

In [1]:
#Online instance only.
from google.colab import drive #drive.flush_and_unmount()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with sqlite3.connect('./raw/all-the-news.db') as db:
    raw = pd.read_sql_query('SELECT * from longform',db)

In [0]:
df2 = pd.read_pickle('./raw/all-the-news.pkl')
import bz2

In [0]:
df2.to_pickle('./raw/all-the-news.pkl.xz',compression='xz')


In [0]:
df = pd.read_pickle('./raw/all-the-news.pkl.xz')

In [0]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.dropna(subset=['content'])
df = df[df.content.str.len() != 0]

In [0]:
df = df[df.content.str.len() >= 20]

In [0]:
remove = ('No.',',','[ ]',"['']",'[, ]','[] ,','“','[]')
def filterWeird(x):
    return all(not chk == x for chk in remove)
df = df[df.content.apply(filterWeird)]

In [0]:
df = df.rename(columns={'content':'text'})

In [0]:
import unicodedata
#normally also helps remove foreign languages
removables = ['AD']
def clean(x):
    x.title = ' '.join(w for w in unicodedata.normalize('NFKD',x.title.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    x.text = ' '.join(w for w in unicodedata.normalize('NFKD',x.text.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    if len(x.title) < 1 or len(x.text.split()) < 150:
        x.title = 'None'
        x.text = 'None'
    return x

In [0]:
df = df.dropna(subset=('text',))

In [0]:
df = df.apply(clean,axis=1)[df.text != 'None']

In [0]:
df = df.drop_duplicates(subset=('text',))

In [20]:
if not 'uuid' in df:
    import uuid
    df = df.assign(uuid=tuple(uuid.uuid4() for x in range(len(df))))
    print('done')

done


In [0]:
df.text = df.text.str.lower()
df.title = df.title.str.lower()

In [22]:
subset_impt = ('text',)
df = df.drop_duplicates(subset=subset_impt).dropna(subset=subset_impt).drop_duplicates(subset=subset_impt).dropna(subset=subset_impt).apply(clean,axis=1)[df.text != 'None'].drop_duplicates(subset=subset_impt).dropna(subset=subset_impt).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
df = df[df.text != 'None']

In [28]:
set(df[df.text.str.len() < 100].text)

set()

In [31]:
df.groupby('publication').count()

,id,title,author,date,text,year,month,category,digital,section,url,uuid
publication,,,,,,,,,,,,
Atlantic,7204,7204,6253,7204,7204,7204,7204,7204,7204,0,0,7204
Breitbart,21337,21337,21337,21337,21337,21337,21337,21337,21337,0,0,21337
Business Insider,5396,5396,4017,5396,5396,5396,5396,5396,5396,0,0,5396
Buzzfeed News,4516,4516,4516,4516,4516,4516,4516,4516,4516,4516,4516,4516
CNN,11041,11041,6856,11041,11041,11041,11041,11041,11041,0,0,11041
Fox News,4126,4126,1110,4126,4126,4126,4126,4126,4126,4124,4124,4126
Guardian,8590,8590,7223,8571,8590,8571,8571,8590,8590,8590,8590,8590
Los Angeles Times,8585,8585,8576,8574,8585,8574,8574,8585,8585,30,30,8585
NPR,11750,11750,11466,11750,11750,11750,11750,11750,11750,7665,11750,11750


In [0]:
df = df.drop(columns =['id','author','year','month','digital'])

In [0]:
df.to_pickle('./raw/all-the-news.pkl.xz',compression='xz')